# Test of the clasification pipeline

In [1]:
import os

# Function to change to the parent directory
def change_to_parent_directory():
    # Check if the directory has already been changed
    if not os.environ.get('DIR_CHANGED'):
        try:
            current_dir = os.path.dirname(os.path.abspath(__file__))
        except NameError:
            current_dir = os.getcwd()
        parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
        os.chdir(parent_dir)
        os.environ['DIR_CHANGED'] = '1'
        print(f"Current working directory changed to: {os.getcwd()}")
    else:
        print("Directory has already been changed.")

# Call the function to change the working directory
change_to_parent_directory()

Current working directory changed to: /home/ihranicky/git/html-domainradar-clf


## Optional: Create testing dataset
Note: If you want do to this, set create_test_parquet to **True**

In [13]:
create_test_parquet = False

if create_test_parquet:
    import pandas as pd
    import numpy as np

    # List of input Parquet files along with their maximum rows and desired labels
    #input_files = [
    #    {'file': 'testdata/2405_clftest_benign_filtered.parquet', 'max_rows': 4000, 'label': 'benign'},
    #    {'file': 'testdata/2405_clftest_phishing_filtered.parquet', 'max_rows': 480, 'label': 'phishing'},
    #    {'file': 'testdata/2405_clftest_malware_filtered.parquet', 'max_rows': 292, 'label': 'malware'},
    #    {'file': 'testdata/dga_2310.parquet', 'max_rows': 300, 'label': 'dga'},
    #]
    
    #input_files = [
    #    {'file': 'testdata/2405_clftest_benign_filtered.parquet', 'max_rows': 300, 'label': 'benign'},
    #    {'file': 'testdata/2405_clftest_phishing_filtered.parquet', 'max_rows': 308, 'label': 'phishing'},
    #    {'file': 'testdata/2405_clftest_malware_filtered.parquet', 'max_rows': 292, 'label': 'malware'},
    #    {'file': 'testdata/dga_2310.parquet', 'max_rows': 300, 'label': 'dga'},
    #]

    input_files = [
        {'file': 'testdata/benign_2312_HTML.parquet', 'max_rows': 150, 'label': 'benign'},
        {'file': 'testdata/phishing_2406_strict_HTML.parquet', 'max_rows': 150, 'label': 'phishing'},
    ]


    # Number of rows to select in total
    #n_rows = 5072
    n_rows = 300
    
    # Read the first file to get the initial columns and create the first dataframe
    first_file_info = input_files[0]
    combined_df = pd.read_parquet(first_file_info['file'])
    
    # Limit the number of rows if necessary for the first file
    if len(combined_df) > first_file_info['max_rows']:
        combined_df = combined_df.sample(n=first_file_info['max_rows'], random_state=1)
    
    # Overwrite the "label" column with the specified label for the first file
    combined_df['label'] = first_file_info['label']
    
    # Get the columns from the first dataframe
    all_columns = combined_df.columns.tolist()

    # Process the remaining files
    for file_info in input_files[1:]:
        df = pd.read_parquet(file_info['file'])
        
        # Limit the number of rows if necessary
        if len(df) > file_info['max_rows']:
            df = df.sample(n=file_info['max_rows'], random_state=1)
        
        # Overwrite the "label" column with the specified label
        df['label'] = file_info['label']
        
        # Ensure all columns from the first dataframe are present
        for col in all_columns:
            if col not in df.columns:
                df[col] = None
        
        # Align the dataframe to the columns of the first dataframe
        df = df[all_columns]
        
        # Append the dataframe to the combined dataframe
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    # Randomly select n_rows rows from the combined DataFrame
    selected_rows = combined_df.sample(n=n_rows, random_state=1)  # random_state for reproducibility

    selected_rows = selected_rows.drop(columns=['label_x', 'label_y'], errors='ignore')
    
    # Save the selected rows to a new Parquet file
    selected_rows.to_parquet('testdata/ver_html_test.parquet')


## Run classification

In [9]:
# Specify the parquet file with the dataset for classification
test_dataset = 'testdata/ver_html_test.parquet'

# Number of domain names to classify with each run of the pipeline (0 = classify all)
CHUNK_SIZE = 30

In [4]:
import pandas as pd
import numpy as np
import shap
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from classifiers.pipeline import Pipeline
from classifiers.options import PipelineOptions
#import matplotlib.pyplot as plt

# Initialize the classification pipeline
clf_options = PipelineOptions()
clf = Pipeline(clf_options)

# Read the input parquet file
input_df = pd.read_parquet(test_dataset)

# Function to map labels to 'benign' or 'malign'
def map_label(label):
    if label == 'benign':
        return 'benign'
    else:
        return f'malign ({label})'

# Function to convert labels to binary classes
def binary_label(label):
    return 'negative' if label == 'benign' else 'positive'

# Apply label mapping
input_df['mapped_label'] = input_df['label'].apply(map_label)
input_df['binary_label'] = input_df['label'].apply(binary_label)

# Ensure SHAP JavaScript initialization
#shap.initjs()

# Determine the number of chunks
num_chunks = (len(input_df) + CHUNK_SIZE - 1) // CHUNK_SIZE if CHUNK_SIZE > 0 else 1

# Initialize counters for overall statistics
total_true_labels = []
total_pred_labels = []

# Format string for aligned output
header_format_str = "{:<3} | {:<50} | {:<18} | {:<10} | {:<10}"
data_format_str = "{:<3} | {:<50} | {:<18} | {:<10} | {:.6f}"

# Process the dataframe in chunks
for i in range(num_chunks):
    if CHUNK_SIZE > 0:
        start_idx = i * CHUNK_SIZE
        end_idx = start_idx + CHUNK_SIZE
        chunk_df = input_df[start_idx:end_idx]
    else:
        chunk_df = input_df

    # Perform your classification or processing on the working_df here
    print(f"===== Processing chunk {i+1}/{num_chunks} =====")

    chunk_without_label = chunk_df.drop(columns=['label', 'mapped_label', 'binary_label']) # Label should not be known to classifiers
    chunk_results = clf.classify_domains(chunk_without_label)

    # Collect predictions and true labels
    true_labels = chunk_df['binary_label'].values
    pred_labels = []
    for result in chunk_results:
        pred_label = 'negative' if result['aggregate_probability'] < 0.5 else 'positive'
        pred_labels.append(pred_label)

    # Update overall statistics
    total_true_labels.extend(true_labels)
    total_pred_labels.extend(pred_labels)

    # Display header for results
    print(header_format_str.format("Res", "Domain Name", "Actual Label", "Predicted", "Probability"))

    # Display results for each domain
    for idx, result in enumerate(chunk_results):
        actual_label = chunk_df.iloc[idx]['mapped_label']
        predicted_label = 'benign' if pred_labels[idx] == 'negative' else 'malign'
        domain_name = result['domain_name']
        aggregate_probability = result['aggregate_probability']
        status = "OK" if pred_labels[idx] == true_labels[idx] else "ER"
        print(data_format_str.format(status, domain_name[:50], actual_label, predicted_label, aggregate_probability))
        
        # Run debug_domain method for misclassified domains
        #if status == "ER":
            #print(f"Debugging misclassified domain: {domain_name}")
            #ndf_data = clf.pp.df_to_NDF(chunk_df, "phishing")  # Convert to NDF
            #debug_data = clf.clf_phishing_cnn.debug_domain(domain_name, ndf_data, chunk_df, n_top_features=10)
            
            # Print out the top n feature importances and values for each classifier
            #for classifier, data in debug_data.items():
            #    print(f"\nClassifier: {classifier}")
            #    print(f"Top {len(data['top_features'])} features for domain '{domain_name}':")
            #    for feature_info in data['top_features']:
            #        print(f"Feature: {feature_info['feature']}, Value: {feature_info['value']}, SHAP Value: {feature_info['shap_value']}")
                
                # Display the force plot for phishing_lgbm
                #if classifier == "phishing_cnn":
                #    base_value, shap_values, domain_row = data['force_plot_data']
                #    shap.force_plot(base_value, shap_values, domain_row)
                #    plt.show()

    # Calculate metrics for the current chunk
    accuracy = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels, pos_label='positive', average='binary')
    recall = recall_score(true_labels, pred_labels, pos_label='positive', average='binary')
    f1 = f1_score(true_labels, pred_labels, pos_label='positive', average='binary')

    tn, fp, fn, tp = confusion_matrix(true_labels, pred_labels, labels=['negative', 'positive']).ravel()
    false_positives = fp
    false_negatives = fn
    total_positives = tp + fp
    total_negatives = tn + fn

    fp_ratio = (false_positives / total_positives) if total_positives > 0 else 0
    fn_ratio = (false_negatives / total_negatives) if total_negatives > 0 else 0

    print(f"Chunk {i+1}/{num_chunks} metrics:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"False Positives: {false_positives} ({fp_ratio * 100:.2f}%)")
    print(f"False Negatives: {false_negatives} ({fn_ratio * 100:.2f}%)")
    print(f"===== Chunk {i+1}/{num_chunks} completed. =====")

# Calculate overall metrics
overall_accuracy = accuracy_score(total_true_labels, total_pred_labels)
overall_precision = precision_score(total_true_labels, total_pred_labels, pos_label='positive', average='binary')
overall_recall = recall_score(total_true_labels, total_pred_labels, pos_label='positive', average='binary')
overall_f1 = f1_score(total_true_labels, total_pred_labels, pos_label='positive', average='binary')

overall_tn, overall_fp, overall_fn, overall_tp = confusion_matrix(total_true_labels, total_pred_labels, labels=['negative', 'positive']).ravel()
overall_false_positives = overall_fp
overall_false_negatives = overall_fn
overall_total_positives = overall_tp + overall_fp
overall_total_negatives = overall_tn + overall_fn

overall_fp_ratio = (overall_false_positives / overall_total_positives) if overall_total_positives > 0 else 0
overall_fn_ratio = (overall_false_negatives / overall_total_negatives) if overall_total_negatives > 0 else 0

print("Overall metrics:")
print(f"Overall Accuracy: {overall_accuracy}")
print(f"Overall Precision: {overall_precision}")
print(f"Overall Recall: {overall_recall}")
print(f"Overall F1 Score: {overall_f1}")
print(f"Overall False Positives: {overall_false_positives} ({overall_fp_ratio * 100:.2f}%)")
print(f"Overall False Negatives: {overall_false_negatives} ({overall_fn_ratio * 100:.2f}%)")


/home/ihranicky/git/domainradar-clf/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-16 14:23:41.524474: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


CNN model created
===== Processing chunk 1/10 =====
Res | Domain Name                                        | Actual Label       | Predicted  | Probability
OK  | nftyjsekyu1.pages.dev                              | malign (phishing)  | malign     | 0.679495
OK  | caldiae.gal                                        | benign             | benign     | 0.000000
OK  | servismail6.figomif911.workers.dev                 | malign (phishing)  | malign     | 0.675342
OK  | ftmjy.blogspot.sn                                  | malign (phishing)  | malign     | 0.659162
OK  | hamster.desi                                       | benign             | benign     | 0.000000
OK  | southerncoast.realtor                              | benign             | benign     | 0.000000
OK  | aol-100904.weeblysite.com                          | malign (phishing)  | malign     | 0.668649
OK  | mmo.org.hk                                         | benign             | benign     | 0.000000
OK  | reagan-mail-109004.we

Res | Domain Name                                        | Actual Label       | Predicted  | Probability
OK  | www.hledampraci.cz                                 | benign             | benign     | 0.000000
OK  | idvtnkyboupid.net                                  | malign (dga)       | malign     | 0.631900
ER  | outlander-lb.shop                                  | benign             | malign     | 0.504027
OK  | bafybeidvkboe27s67yrcdr74gfgwh26fhiidh2l53rmh55mvg | malign (phishing)  | malign     | 0.704089
OK  | poly.skoda-auto.cz                                 | benign             | benign     | 0.000000
OK  | justt-watch-now.xyz                                | malign (malware)   | malign     | 0.739149
OK  | starlancercenter.com                               | malign (malware)   | malign     | 0.754605
OK  | ayicsu.com                                         | malign (dga)       | malign     | 0.600150
OK  | item-list22119.atwebpages.com                      | malign (phishing)  |

Res | Domain Name                                        | Actual Label       | Predicted  | Probability
OK  | 1xgkpcuwpf99f34050011gky86.org                     | malign (dga)       | malign     | 0.656312
ER  | transportkazan.ru                                  | benign             | malign     | 0.549271
OK  | agyncivbkee.work                                   | malign (dga)       | malign     | 0.579991
OK  | www.sposobzyciowy.click                            | malign (malware)   | malign     | 0.854492
OK  | jxjydzlvmzk67eqnygxnqaypui15l18d60o61.com          | malign (dga)       | malign     | 0.689876
OK  | c93ec95a80b21090fd7e34e305abcca9.nitwos.pl         | malign (malware)   | malign     | 0.852340
ER  | fjz.fm                                             | malign (dga)       | benign     | 0.000000
OK  | taghleefpack.ignacyklaudiusz.pl                    | malign (malware)   | malign     | 0.687742
OK  | cwzqkxrvlmifrqq.com                                | malign (dga)       |

Res | Domain Name                                        | Actual Label       | Predicted  | Probability
OK  | rutabagacalculusar.pro                             | malign (malware)   | malign     | 0.859254
OK  | nakitel.com                                        | benign             | benign     | 0.000000
OK  | pixel.jobteaser.com                                | benign             | benign     | 0.000000
OK  | 19rgz8h16w8khxhfcv6p10vhsam.com                    | malign (dga)       | malign     | 0.657988
OK  | aspb1.cdn.asset.aparat.com                         | benign             | benign     | 0.000000
OK  | www.technikapoznani.cz                             | benign             | benign     | 0.000000
ER  | www.fahuwk.com                                     | benign             | malign     | 0.728230
OK  | pacificobdpec.ac-page.com                          | malign (phishing)  | malign     | 0.692917
OK  | pub-e5c2c5311c064a7bb152dc2d6b4b0d49.r2.dev        | malign (malware)   |

Res | Domain Name                                        | Actual Label       | Predicted  | Probability
OK  | bt-business-5-22-2024.webflow.io                   | malign (phishing)  | malign     | 0.693447
OK  | hmxfsdrvxfq.ga                                     | malign (dga)       | malign     | 0.652697
ER  | promptly-complete-bay-renewable.trycloudflare.com  | malign (phishing)  | benign     | 0.000000
OK  | bt-business-5-21-2024.webflow.io                   | malign (phishing)  | malign     | 0.697082
OK  | 1jiltjpvihzbmfn5stckufmtv.com                      | malign (dga)       | malign     | 0.659230
OK  | www.olomoucdnes.cz                                 | benign             | benign     | 0.000000
OK  | inncome.manasan.top                                | malign (malware)   | malign     | 0.627488
OK  | mehakramzan.github.io                              | malign (phishing)  | malign     | 0.712318
OK  | www.graphpad.com                                   | benign             |

Res | Domain Name                                        | Actual Label       | Predicted  | Probability
OK  | pxqmxxmkzdbddfiedgqbx.net                          | malign (dga)       | malign     | 0.653313
OK  | hw-sh-pcdn-17.biliapi.net                          | benign             | benign     | 0.000000
OK  | exchange.cryptex.net                               | benign             | benign     | 0.000000
ER  | smalldawn.hn                                       | malign (dga)       | benign     | 0.000000
ER  | 0x-code.financialmarketsworld.com                  | malign (malware)   | benign     | 0.463569
ER  | foxmodeq.com                                       | benign             | malign     | 0.738716
OK  | vsosyyuxlvvfpxr.org                                | malign (dga)       | malign     | 0.666033
OK  | ridertools.metrarail.com                           | benign             | benign     | 0.000000
OK  | urbanabodes.homes                                  | malign (malware)   |

Res | Domain Name                                        | Actual Label       | Predicted  | Probability
OK  | attnet-106864.weeblysite.com                       | malign (phishing)  | malign     | 0.704819
OK  | www.windguru.cz                                    | benign             | benign     | 0.000000
ER  | waitthere.net                                      | malign (dga)       | benign     | 0.282252
OK  | www.fibstaff.com                                   | malign (malware)   | malign     | 0.847769
OK  | 0503c3bdb2699f8d02db98d87d97f4ea.loftx.pl          | malign (malware)   | malign     | 0.739372
ER  | wepik.com                                          | malign (phishing)  | benign     | 0.000000
OK  | temperate-kangaroo-kw0hgq.mystrikingly.com         | malign (phishing)  | malign     | 0.640199
OK  | pikolmnfjihjuu.com                                 | malign (dga)       | malign     | 0.653120
OK  | www.habitatetjardin.com                            | benign             |

## Optional: Generate preliminary results for training the final aggregation classifier

In [10]:
import pandas as pd
import numpy as np
from classifiers.pipeline import Pipeline

# Initialize the classification pipeline
clf = Pipeline()

# Read the input parquet file
input_df = pd.read_parquet(test_dataset)

input_df = input_df.sample(frac=1).reset_index(drop=True)

#preliminary_results_df = clf.generate_preliminary_results(input_df, output_file="test_preliminary_results.parquet")
preliminary_results_df = clf.generate_preliminary_results(input_df, add_final=False)

CNN model created


In [11]:
pd.reset_option('display.max_rows')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)
pd.options.display.float_format = '{:.6f}'.format
preliminary_results_df

,domain_name,dns_available,dns_nonzero,tls_available,tls_nonzero,ip_available,ip_nonzero,rdap_available,rdap_nonzero,geo_available,geo_nonzero,phishing_cnn_result,phishing_lgbm_result,phishing_xgboost_result,phishing_deepnn_result,phishing_dns_nn_result,phishing_rdap_nn_result,phishing_geo_nn_result,phishing_ip_nn_result,phishing_html_lgbm_result,malware_lgbm_result,malware_xgboost_result,malware_deepnn_result,malware_dns_nn_result,malware_rdap_nn_result,malware_geo_nn_result,malware_ip_nn_result,dga_binary_deepnn_result,dga_binary_lgbm_result,phishing_sum,phishing_avg,phishing_prod,malware_sum,malware_avg,malware_prod,dga_binary_sum,dga_binary_avg,dga_binary_prod,total_sum,total_avg,total_prod,label
0,publicaciones.claeh.edu.uy,0.650000,0.300000,0.833333,0.291667,1.000000,0.625000,0.833333,0.291667,1.000000,0.722222,0.000048,0.000510,0.000595,0.000484,0.019368,0.003055,0.390085,0.232554,0.000000,0.000449,0.002364,0.000004,0.085267,0.000785,0.307506,0.328502,0.000000,0.000000,0.646699,0.071855,0.000000,0.724879,0.103554,0.000000,0.000000,0.000000,0.000000,1.371577,0.076199,0.000000,benign
1,strategicgames.com.au,0.950000,0.600000,0.833333,0.291667,1.000000,1.000000,0.875000,0.583333,1.000000,1.000000,0.010980,0.001454,0.001453,0.018853,0.016694,0.195629,0.024833,0.072707,1.000000,0.001910,0.006233,0.000016,0.186587,0.090949,0.003227,0.001105,0.000001,0.000002,1.342602,0.149178,0.000000,0.290027,0.041432,0.000000,0.000002,0.000001,0.000000,1.632631,0.090702,0.000000,phishing
2,festiwal.uni.opole.pl,0.600000,0.225000,0.833333,0.291667,1.000000,0.375000,0.833333,0.208333,1.000000,0.666667,0.000002,0.000203,0.000430,0.000014,0.009321,0.000872,0.007026,0.054764,0.000000,0.000325,0.002285,0.000002,0.050616,0.000963,0.058723,0.005198,0.000000,0.000000,0.072632,0.008070,0.000000,0.118111,0.016873,0.000000,0.000000,0.000000,0.000000,0.190744,0.010597,0.000000,benign
3,bafkreiht4h7zhmfirkjoq2sq6cir64gijftvdskjup54h...,0.950000,0.600000,0.833333,0.291667,1.000000,0.625000,1.000000,0.833333,1.000000,0.722222,0.993279,0.724641,0.958058,0.808913,0.997240,0.964806,0.390085,0.867334,1.000000,0.001416,0.002883,0.000221,0.005848,0.024984,0.307506,0.196051,0.000000,0.000014,7.704357,0.856040,0.181582,0.538910,0.076987,0.000000,0.000014,0.000007,0.000000,8.243280,0.457960,0.000000,phishing
4,recycle.green,1.000000,0.700000,1.000000,0.750000,1.000000,1.000000,1.000000,0.833333,1.000000,1.000000,0.001489,0.000257,0.000400,0.001420,0.022805,0.004532,0.047657,0.048546,0.000000,0.000064,0.002233,0.000000,0.142886,0.000696,0.000621,0.000480,0.000087,0.000018,0.127106,0.014123,0.000000,0.146981,0.020997,0.000000,0.000105,0.000053,0.000000,0.274192,0.015233,0.000000,benign
5,floo.bar,1.000000,0.725000,0.833333,0.291667,1.000000,1.000000,1.000000,0.916667,1.000000,1.000000,0.002183,0.000470,0.000497,0.001044,0.008241,0.004243,0.028413,0.042594,0.000000,0.000058,0.002233,0.000000,0.107033,0.000013,0.002643,0.000823,0.010728,0.000162,0.087685,0.009743,0.000000,0.112805,0.016115,0.000000,0.010890,0.005445,0.000002,0.211380,0.011743,0.000000,benign
6,iris.gmbh,1.000000,0.775000,0.833333,0.291667,1.000000,1.000000,1.000000,0.708333,1.000000,1.000000,0.001824,0.000350,0.000567,0.000184,0.003431,0.004263,0.184730,0.038991,0.000000,0.000052,0.002233,0.000000,0.019034,0.000003,0.003055,0.000070,0.001707,0.000016,0.234340,0.026038,0.000000,0.024447,0.003492,0.000000,0.001723,0.000861,0.000000,0.260510,0.014473,0.000000,benign
7,christopherstalbergmd.com,0.475000,0.000000,0.833333,0.291667,0.625000,0.000000,1.000000,0.291667,1.000000,0.000000,1.000000,0.004452,0.002544,0.011773,0.184629,0.029043,0.387495,0.414404,1.000000,0.005589,0.008462,0.000157,0.394762,0.020817,0.604780,0.443381,0.575031,0.921792,3.034341,0.337149,0.000000,1.477949,0.211136,0.000000,1.496822,0.748411,0.530059,6.009112,0.333840,0.000000,phishing
8,oaisjbbsay.com,0.900000,0.550000,0.833333,0.291667,1.000000,0.875000,1.000000,0.708333,1.000000,1.000000,0.993279,0.799467,0.900177,0.976299,0.167700,

In [16]:
pd.set_option('display.max_rows', None)
preliminary_results_df[preliminary_results_df["label"].isin(["benign", "phishing"])][["domain_name", "label", "phishing_cnn_result", "phishing_xgboost_result", "phishing_html_lgbm_result", "phishing_lgbm_result"]]

,domain_name,label,phishing_cnn_result,phishing_xgboost_result,phishing_html_lgbm_result,phishing_lgbm_result
0,publicaciones.claeh.edu.uy,benign,0.000048,0.000595,0.000000,0.000510
1,strategicgames.com.au,phishing,0.010980,0.001453,1.000000,0.001454
2,festiwal.uni.opole.pl,benign,0.000002,0.000430,0.000000,0.000203
3,bafkreiht4h7zhmfirkjoq2sq6cir64gijftvdskjup54h...,phishing,0.993279,0.958058,1.000000,0.724641
4,recycle.green,benign,0.001489,0.000400,0.000000,0.000257
5,floo.bar,benign,0.002183,0.000497,0.000000,0.000470
6,iris.gmbh,benign,0.001824,0.000567,0.000000,0.000350
7,christopherstalbergmd.com,phishing,1.000000,0.002544,1.000000,0.004452
8,oaisjbbsay.com,phishing,0.993279,0.900177,1.000000,0.799467
9,riskexplorer.technology,benign,0.000016,0.000420,0.000000,0.000343


In [8]:
preliminary_results_df[["domain_name", "label", "malware_deepnn_result", "malware_lgbm_result", "malware_xgboost_result"]]

,domain_name,label,malware_deepnn_result,malware_lgbm_result,malware_xgboost_result
0,olkano.com,dga,0.001556,0.013970,0.047478
1,brandstorm.loreal.com,benign,0.000122,0.000514,0.002626
2,wytypowany-zwyciezca.com,malware,0.759562,0.383536,0.286979
3,oopt.fr,malware,0.122850,0.277118,0.445704
4,16chan.org,malware,0.627538,0.708378,0.589509
5,down.t0kbnpobket.biz,phishing,0.000658,0.010174,0.005187
6,bolsadetrabajosgt.com,phishing,0.000000,0.000067,0.002233
7,www.sprawozdania24.click,malware,0.849272,0.677257,0.907050
8,discord.writemall.top,phishing,0.000252,0.000411,0.002364
9,patasblancasvet.com,malware,0.794451,0.881981,0.884218


In [9]:
preliminary_results_df[preliminary_results_df["label"].isin(["benign", "malware"])][
    ["domain_name", "label", "malware_deepnn_result", "malware_lgbm_result", "malware_xgboost_result", "malware_dns_nn_result", "malware_rdap_nn_result", "malware_geo_nn_result"]
]


,domain_name,label,malware_deepnn_result,malware_lgbm_result,malware_xgboost_result,malware_dns_nn_result,malware_rdap_nn_result,malware_geo_nn_result
1,brandstorm.loreal.com,benign,0.000122,0.000514,0.002626,0.184353,0.001344,0.399883
2,wytypowany-zwyciezca.com,malware,0.759562,0.383536,0.286979,0.775626,0.021642,0.399883
3,oopt.fr,malware,0.122850,0.277118,0.445704,0.070304,0.024303,0.209001
4,16chan.org,malware,0.627538,0.708378,0.589509,0.941982,0.049690,0.605018
7,www.sprawozdania24.click,malware,0.849272,0.677257,0.907050,0.589228,0.398529,0.399883
9,patasblancasvet.com,malware,0.794451,0.881981,0.884218,0.888655,0.061971,0.689959
10,www.ipop.at,benign,0.000021,0.000861,0.003839,0.032429,0.175987,0.129151
13,vurbont.xyz,malware,0.889243,0.986667,0.955447,0.398776,0.310197,0.399883
14,srebny23.aleksytymon.com.pl,malware,0.000637,0.001621,0.002364,0.335020,0.000711,0.399883
15,tools.pinpoll.com,benign,0.000008,0.000501,0.002408,0.076731,0.047117,0.016066


In [10]:
preliminary_results_df[preliminary_results_df["label"].isin(["benign", "malware"])][
    ["domain_name", "label", "malware_deepnn_result", "malware_dns_nn_result", "malware_rdap_nn_result", "malware_geo_nn_result", "malware_ip_nn_result"]
]


,domain_name,label,malware_deepnn_result,malware_dns_nn_result,malware_rdap_nn_result,malware_geo_nn_result,malware_ip_nn_result
1,brandstorm.loreal.com,benign,0.000122,0.184353,0.001344,0.399883,0.548636
2,wytypowany-zwyciezca.com,malware,0.759562,0.775626,0.021642,0.399883,0.212463
3,oopt.fr,malware,0.122850,0.070304,0.024303,0.209001,0.450995
4,16chan.org,malware,0.627538,0.941982,0.049690,0.605018,0.279973
7,www.sprawozdania24.click,malware,0.849272,0.589228,0.398529,0.399883,0.452909
9,patasblancasvet.com,malware,0.794451,0.888655,0.061971,0.689959,0.330984
10,www.ipop.at,benign,0.000021,0.032429,0.175987,0.129151,0.006270
13,vurbont.xyz,malware,0.889243,0.398776,0.310197,0.399883,0.462252
14,srebny23.aleksytymon.com.pl,malware,0.000637,0.335020,0.000711,0.399883,0.637277
15,tools.pinpoll.com,benign,0.000008,0.076731,0.047117,0.016066,0.005857


In [11]:
#print(preliminary_results_df['malware_geo_nn_result'].unique())
#for value in preliminary_results_df['malware_geo_nn_result'].unique():
#    print(value)

In [12]:
preliminary_results_df[["label", "dga_binary_deepnn_result", "dga_binary_lgbm_result"]]

,label,dga_binary_deepnn_result,dga_binary_lgbm_result
0,dga,0.922759,0.195645
1,benign,0.000000,0.000002
2,malware,0.156716,0.007199
3,malware,0.136516,0.001036
4,malware,0.027879,0.001080
5,phishing,0.000001,0.001439
6,phishing,0.482309,0.006404
7,malware,0.000000,0.010450
8,phishing,0.000000,0.000006
9,malware,0.226647,0.003000


In [13]:
preliminary_results_df[["domain_name", "label", "phishing_cnn_result", "phishing_deepnn_result", "phishing_lgbm_result", "phishing_xgboost_result", "phishing_dns_nn_result", "phishing_rdap_nn_result"]]

,domain_name,label,phishing_cnn_result,phishing_deepnn_result,phishing_lgbm_result,phishing_xgboost_result,phishing_dns_nn_result,phishing_rdap_nn_result
0,olkano.com,dga,0.993279,0.000003,0.000610,0.000632,0.184629,0.020158
1,brandstorm.loreal.com,benign,0.012323,0.000363,0.004557,0.001432,0.370768,0.004173
2,wytypowany-zwyciezca.com,malware,0.017531,0.851645,0.023220,0.003387,0.235347,0.003177
3,oopt.fr,malware,0.001619,0.006011,0.007934,0.021717,0.022058,0.006285
4,16chan.org,malware,0.080616,0.004772,0.019884,0.053516,0.027071,0.025906
5,down.t0kbnpobket.biz,phishing,0.972836,0.999845,0.931129,0.930527,0.079166,0.243867
6,bolsadetrabajosgt.com,phishing,0.963146,0.994740,0.032518,0.115738,0.023512,0.003566
7,www.sprawozdania24.click,malware,0.166446,0.974665,0.025775,0.063857,0.268165,0.006818
8,discord.writemall.top,phishing,0.514197,0.897723,0.313745,0.154308,0.153307,0.018440
9,patasblancasvet.com,malware,0.176411,0.966612,0.016730,0.104643,0.541051,0.017393


In [14]:
preliminary_results_df[["domain_name", "label", "malware_deepnn_result", "malware_lgbm_result", "malware_xgboost_result"]]

,domain_name,label,malware_deepnn_result,malware_lgbm_result,malware_xgboost_result
0,olkano.com,dga,0.001556,0.013970,0.047478
1,brandstorm.loreal.com,benign,0.000122,0.000514,0.002626
2,wytypowany-zwyciezca.com,malware,0.759562,0.383536,0.286979
3,oopt.fr,malware,0.122850,0.277118,0.445704
4,16chan.org,malware,0.627538,0.708378,0.589509
5,down.t0kbnpobket.biz,phishing,0.000658,0.010174,0.005187
6,bolsadetrabajosgt.com,phishing,0.000000,0.000067,0.002233
7,www.sprawozdania24.click,malware,0.849272,0.677257,0.907050
8,discord.writemall.top,phishing,0.000252,0.000411,0.002364
9,patasblancasvet.com,malware,0.794451,0.881981,0.884218


In [15]:
preliminary_results_df[["domain_name", "label", "badness_probability"]]

KeyError: "['badness_probability'] not in index"

In [ ]:
preliminary_results_df